# SEIRHVD Data Fit

This Jupyter notebooks implements the SEIRHVD4 model for fitting the actual Chilean data in order to make projections in bed usage, amount of infected and deaths. 
This implements a single instance to facilitate understanding the model prior to run multiple data fittings at a time.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from numpy import linalg as LA

import platform
OS = platform.system()

import matplotlib.pyplot as plt
if OS == 'Linux':    
    %matplotlib tk
    print('Linux')
elif OS == 'Windows':
    %matplotlib qt
    print('Windows')
else:
    print('OS not detected :-|')

import sys
from pathlib import Path
sys.path.insert(1, '../src/SEIRHVD/')
sys.path.insert(1, '../src/utils/')
sys.path.insert(1, 'src/SEIRHVD/')
sys.path.insert(1, 'src/utils/')

from class_SEIRHUVD4 import SEIRHVD 
from Quarantine import Quarantine
from importdata import ImportData

## Time Variables
Declare Initial date for simulating and data fitting

In [ ]:
initdate = datetime(2020,5,15)
initdate2 = datetime(2020,8,24)
currentdate = datetime.now()
currentday = (currentdate - initdate).days

## Import Data
To import data we use an ImportData object which is initialized with the region's cut and the initial date. Meanwhile we will adjust this data for regions, then I'll add Heath sector as the minimum territorial unit for SEIRHVD models.

In [ ]:
tstate = '13'

# Import Data
RM = ImportData(tstate=tstate,initdate = initdate)
RM.importdata()

## Simulation Parameters

In [ ]:
# Simulation parameters
tsimtot = 1000

tsim1 = (initdate2 - initdate).days 
tsim2 = tsimtot-tsim1

beta = 0.16
mu = 0.8
k = 1

SeroPrevFactor1=0.06
SeroPrevFactor2=0.025

expinfection=0 # Proportion in which the exposed infect - 0: nothing, 1: equally as Infected


## Quarantines
Build Quarantine Object:
 
     Q = Quarantine(rem_mob,max_mob=max_mob,qp=0,iqt=0,fqt=1000,movfunct = 'once')
     alpha = Q.alpha
     
     Plot Quarantine dynamics:
         Q.plot()

In [ ]:
# Quarantines 
max_mob = 0.55
rem_mob = 0.45
t_end_date=datetime(2020,8,25)
t_end=(t_end_date-initdate).days

alpha = Quarantine(rem_mob,max_mob,qp=0,iqt=0,fqt=t_end).alpha
alpha = Quarantine(rem_mob).alpha

## Underreport
Fraction of Infected detected/reported  
    - Imi_det: fraction of Mild detected
    - Imi_as: fraction of asymptomatic detected
    
If both are 1, means that all infected are detected so I_det = I

In [ ]:
Imi_det = 1
Ias_det = 0

## Modify State Parameters:
        self.pE_Ias = 0.4# Transicion de Expuesto a Infectado asintomatico
        self.tE_Ias = 5.0

        self.pE_Imi = 0.55#78 # Transicion de Expuesto a Infectado Mild
        self.tE_Imi = 5.0

        self.pE_Icr = 0.01666#8 # Transicion de Expuesto a Infectado critico
        self.tE_Icr = 3.0

        self.pE_Ise = 0.03334#14 # Transicion de Expuesto a Infectado Severo
        self.tE_Ise = 3.0

        self.pIas_R = 1.0   # Transicion de Infectado asintomatico a Recuperado
        self.tIas_R = 10.0 

        self.pImi_R = 1.0  # Transicion de Infectado mild a Recuperado
        self.tImi_R = 15.0

        self.pIse_Hse = 1.0  # Transicion de Infectado serio a Hospitalizado (si no ha colapsado Htot)
        self.tIse_Hse = 3.0 

        self.pIse_D = 1.0  # Transicion de Infectado serio a Hospitalizado (si no ha colapsado Htot)
        self.tIse_D = 3.0         

        self.pIcr_V = 1.0 # Transicion de Hospitalizado critico a Ventilado (si no ha colapsado V)
        self.tIcr_V = 3.0 

        self.pIcr_D = 1.0 # Transicion de Hospitalizado critico a Ventilado (si no ha colapsado V)
        self.tIcr_D = 3.0         

        self.pHse_R = 0.97 # Transicion de Hospitalizado a Hospitalizado Critico (si no ha colapsado Htot)
        self.tHse_R = 11.0

        self.pHse_V = 0.03 # Transicion de Hospitalizado a VMI (si no ha colapsado VMI)
        self.tHse_V = 3.0

        self.pHse_D = 0.03 # Transicion de Hospitalizado a Hospitalizado Critico (si no ha colapsado Htot)
        self.tHse_D = 3.0        

        self.pV_Hout = 0.5 # Mejora de ventilado hospitalizado, transita a Hout
        self.tV_Hout = 15.0

        self.pV_D = 0.5 # VMI to Death
        self.tV_D = 15.0

        self.pHout_R = 1.0 # Mejora del paciente hospitalizado, Hout a R
        self.tHout_R = 4.0

        self.pD_B = 1.0 # Entierros
        self.tD_B = 1.0 

        self.betaD = 0
        self.eta = 0.0 # tasa de perdida de inmunidad (1/periodo)

In [ ]:
pE_Ias = 0.4# Transicion de Expuesto a Infectado asintomatico
tE_Ias = 5.0

pE_Imi = 0.55#78 # Transicion de Expuesto a Infectado Mild
tE_Imi = 5.0

pE_Icr = 0.01666#8 # Transicion de Expuesto a Infectado critico
tE_Icr = 3.0

pE_Ise = 0.03334#14 # Transicion de Expuesto a Infectado Severo
tE_Ise = 3.0

pIas_R = 1.0   # Transicion de Infectado asintomatico a Recuperado
tIas_R = 10.0 

pImi_R = 1.0  # Transicion de Infectado mild a Recuperado
tImi_R = 15.0

pIse_Hse = 1.0  # Transicion de Infectado serio a Hospitalizado (si no ha colapsado Htot)
tIse_Hse = 3.0 

pIse_D = 1.0  # Transicion de Infectado serio a Hospitalizado (si no ha colapsado Htot)
tIse_D = 3.0         

pIcr_V = 1.0 # Transicion de Hospitalizado critico a Ventilado (si no ha colapsado V)
tIcr_V = 3.0 

pIcr_D = 1.0 # Transicion de Hospitalizado critico a Ventilado (si no ha colapsado V)
tIcr_D = 3.0         

pHse_R = 0.97 # Transicion de Hospitalizado a Hospitalizado Critico (si no ha colapsado Htot)
tHse_R = 11.0

pHse_V = 0.03 # Transicion de Hospitalizado a VMI (si no ha colapsado VMI)
tHse_V = 3.0

pHse_D = 0.03 # Transicion de Hospitalizado a Hospitalizado Critico (si no ha colapsado Htot)
tHse_D = 3.0        

pV_Hout = 0.5 # Mejora de ventilado hospitalizado, transita a Hout
tV_Hout = 15.0

pV_D = 0.5 # VMI to Death
tV_D = 15.0

pHout_R = 1.0 # Mejora del paciente hospitalizado, Hout a R
tHout_R = 4.0

## Simulation
Initialize Simulation Object, and run the simulation. 

In [ ]:
simulation1 = SEIRHVD(tsim1,beta,mu,alpha,k=k,SeroPrevFactor=SeroPrevFactor1,expinfection=expinfection,RealIC = RM,Imi_det = Imi_det,Ias_det = Ias_det)

### Setting new params

In [ ]:
simulation1.pE_Ias=pE_Ias
simulation1.tE_Ias=tE_Ias
simulation1.pE_Imi=pE_Imi
simulation1.tE_Imi=tE_Imi
simulation1.pE_Icr=pE_Icr
simulation1.tE_Icr=tE_Icr
simulation1.pE_Ise=pE_Ise
simulation1.tE_Ise=tE_Ise
simulation1.pIas_R=pIas_R
simulation1.tIas_R =tIas_R
simulation1.pImi_R=pImi_R
simulation1.tImi_R =tImi_R
simulation1.pIse_Hse=pIse_Hse
simulation1.tIse_Hse=tIse_Hse
simulation1.pIse_D=pIse_D
simulation1.tIse_D=tIse_D
simulation1.pIcr_V=pIcr_V
simulation1.tIcr_V=tIcr_V
simulation1.pIcr_D=pIcr_D
simulation1.tIcr_D=tIcr_D
simulation1.pHse_R=pHse_R
simulation1.tHse_R =tHse_R
simulation1.pHse_V=pHse_V
simulation1.tHse_V=tHse_V
simulation1.pHse_D=pHse_D
simulation1.tHse_D=tHse_D
simulation1.pV_Hout=pV_Hout
simulation1.tV_Hout =tV_Hout
simulation1.pV_D=pV_D
simulation1.tV_D =tV_D
simulation1.pHout_R=pHout_R
simulation1.tHout_R=tHout_R
simulation1.setnewparams()

## Run simulation

In [ ]:
simulation1.integr_sci(0,tsim1,0.1)

## Simulation 2

In [ ]:
simulation2 = SEIRHVD(tsim2,beta,mu,alpha,k=k,SeroPrevFactor=SeroPrevFactor2,Imi_det = Imi_det,Ias_det = Ias_det,SimIC=simulation1,initdate=initdate2)

In [ ]:
simulation2.pE_Ias=pE_Ias
simulation2.tE_Ias=tE_Ias
simulation2.pE_Imi=pE_Imi
simulation2.tE_Imi=tE_Imi
simulation2.pE_Icr=pE_Icr
simulation2.tE_Icr=tE_Icr
simulation2.pE_Ise=pE_Ise
simulation2.tE_Ise=tE_Ise
simulation2.pIas_R=pIas_R
simulation2.tIas_R =tIas_R
simulation2.pImi_R=pImi_R
simulation2.tImi_R =tImi_R
simulation2.pIse_Hse=pIse_Hse
simulation2.tIse_Hse=tIse_Hse
simulation2.pIse_D=pIse_D
simulation2.tIse_D=tIse_D
simulation2.pIcr_V=pIcr_V
simulation2.tIcr_V=tIcr_V
simulation2.pIcr_D=pIcr_D
simulation2.tIcr_D=tIcr_D
simulation2.pHse_R=pHse_R
simulation2.tHse_R =tHse_R
simulation2.pHse_V=pHse_V
simulation2.tHse_V=tHse_V
simulation2.pHse_D=pHse_D
simulation2.tHse_D=tHse_D
simulation2.pV_Hout=pV_Hout
simulation2.tV_Hout =tV_Hout
simulation2.pV_D=pV_D
simulation2.tV_D =tV_D
simulation2.pHout_R=pHout_R
simulation2.tHout_R=tHout_R
simulation2.setnewparams()

### Run simulation 2

In [ ]:
simulation2.integr_sci(0,tsim2,0.1)

# Analysis

## Grid Plot

In [ ]:
days = 250

In [ ]:
fig, axs = plt.subplots(2, 3)
axs[0,0].set_xlim(0,days)
axs[0,0].scatter(RM.tr,RM.Ir,label='Real Active Data')
axs[0,0].plot(simulation1.t,simulation1.I_det,label='Infected')
axs[0,0].plot(simulation2.t+tsim1,simulation2.I_det,label='Infected')
axs[0,0].set_title('Active Infected')
axs[0,0].axvline(x = t_end, linestyle = 'dotted',color = 'grey')


axs[0,1].set_xlim(0,days)
axs[0,1].plot(simulation2.t+tsim1,simulation2.I_d_det,label='sim2')
axs[0,1].plot(simulation1.t,simulation1.I_d_det,label='sim1')
axs[0,1].scatter(RM.I_d_r_tr,RM.I_d_r,label='Daily Real Data')
axs[0,1].set_title('Daily infected')
axs[0,1].axvline(x = t_end, linestyle = 'dotted',color = 'grey')


axs[0,2].set_xlim(0,days)
axs[0,2].plot(simulation2.t+tsim1,simulation2.I_ac_det,label='sim2')
axs[0,2].plot(simulation1.t,simulation1.I_ac_det,label='sim1')
axs[0,2].scatter(RM.I_ac_r_tr,RM.I_ac_r,label='Accumulated Real Data')
axs[0,2].set_title('Accumulated Infected')
axs[0,2].axvline(x = t_end, linestyle = 'dotted',color = 'grey')


axs[1,0].set_xlim(0,days)
axs[1,0].plot(simulation2.t+tsim1,simulation2.B,label='sim2')
axs[1,0].plot(simulation1.t,simulation1.B,label='sim1')
axs[1,0].scatter(RM.Br_tr,RM.Br,label='Real Data')
axs[1,0].set_title('Acmulated Deaths')
axs[1,0].axvline(x = t_end, linestyle = 'dotted',color = 'grey')


axs[1,1].set_xlim(0,days)
axs[1,1].plot(simulation1.t,simulation1.Hse+simulation1.Hout,label='UCI/UTI Beds')
axs[1,1].plot(simulation2.t+tsim1,simulation2.Hse+simulation2.Hout,label='UCI/UTI Beds')
axs[1,1].scatter(RM.sochimi_tr,RM.Hr,label='Real Data')
axs[1,1].scatter(RM.sochimi_tr,RM.Hr_tot,label='Capacity Data', color = 'red')
axs[1,1].set_title('UCI/UTI Usage')
axs[1,1].axvline(x = t_end, linestyle = 'dotted',color = 'grey')


axs[1,2].set_xlim(0,days)
axs[1,2].plot(simulation2.t+tsim1,simulation2.V,label='VSat sim2')
axs[1,2].plot(simulation1.t,simulation1.V,label='Vsat sim1')
axs[1,2].scatter(RM.sochimi_tr,RM.Vr,label='Real Data')
axs[1,2].scatter(RM.sochimi_tr,RM.Vr_tot,label='Capacity Data',color = 'red')
axs[1,2].set_title('VMI Usage')
axs[1,2].axvline(x = t_end, linestyle = 'dotted',color = 'grey')


#axs[2,1].plot(simulation2.t+tsim1,simulation2.V_D_d,label='V_D sim2')
#axs[2,1].plot(simulation2.t+tsim1,simulation2.Hse_D_d,label='Hse_D sim2')
#axs[2,1].plot(simulation2.t+tsim1,simulation2.Icr_D_d,label='Icr_D sim2')
#axs[2,1].plot(simulation2.t+tsim1,simulation2.Ise_D_d,label='Ise_D sim2')
#axs[2,1].plot(simulation1.t,simulation1.V_D_d,label='V_D sim1')
#axs[2,1].plot(simulation1.t,simulation1.Hse_D_d,label='Hse_D sim1')
#axs[2,1].plot(simulation1.t,simulation1.Icr_D_d,label='Icr_D sim1')
#axs[2,1].plot(simulation1.t,simulation1.Ise_D_d,label='Ise_D sim1')
#axs[2,1].set_title('Death by cause')
#axs[2,1].axvline(x = t_end, linestyle = 'dotted',color = 'grey')


## Single plots

### Daily new Infected

In [ ]:
plt.plot(simulation2.t+tsim1,simulation2.I_d_det,label='sim2')
plt.plot(simulation1.t,simulation1.I_d_det,label='sim1')
plt.legend(loc=0)
plt.title('Daily New Infected')
plt.show()

### Daily Deaths

In [ ]:
plt.plot(simulation2.t+tsim1,simulation2.D,label='sim2')
plt.plot(simulation1.t,simulation1.D,label='sim1')
plt.legend(loc=0)
plt.title('Deaths')
plt.show()

### VMI Usage

In [ ]:
plt.plot(simulation2.t+tsim1,simulation2.V_cap,label='VMI capacity sim2')
plt.plot(simulation1.t,simulation1.V_cap,label='VMI capacity sim1')
plt.plot(simulation1.t,simulation1.V,label='VMI sim1')
plt.plot(simulation2.t+tsim1,simulation2.V,label='VMI sim2')
plt.legend(loc=0)
plt.title('VMI')
plt.scatter(RM.sochimi_tr,RM.Vr,label='Real Data')
plt.scatter(RM.sochimi_tr,RM.Vr_tot,label='Real Data')
plt.show()

### Hospitalization

In [ ]:
plt.plot(simulation2.t+tsim1,simulation2.H_cap,label='sim2')
plt.plot(simulation1.t,simulation1.H_cap,label='sim1')
plt.plot(simulation1.t,simulation1.Hse+simulation1.Hout,label='UCI/UTI Beds')
plt.plot(simulation2.t+tsim1,simulation2.Hse+simulation2.Hout,label='UCI/UTI Beds')
plt.legend(loc=0)
plt.title('Hospitalization')

plt.scatter(RM.sochimi_tr,RM.Hr,label='Real Data')
plt.scatter(RM.sochimi_tr,RM.Hr_tot,label='Real Data')
plt.show()

### Deaths by cause

In [ ]:
plt.plot(simulation2.t+tsim1,simulation2.D,label='sim2')
plt.plot(simulation1.t,simulation1.D,label='sim1')


plt.plot(simulation2.t+tsim1,simulation2.V_D_d,label='V_D sim2')
plt.plot(simulation2.t+tsim1,simulation2.Hse_D_d,label='Hse_D sim2')
plt.plot(simulation2.t+tsim1,simulation2.Icr_D_d,label='Icr_D sim2')
plt.plot(simulation2.t+tsim1,simulation2.Ise_D_d,label='Ise_D sim2')

plt.plot(simulation1.t,simulation1.V_D_d,label='V_D sim1')
plt.plot(simulation1.t,simulation1.Hse_D_d,label='Hse_D sim1')
plt.plot(simulation1.t,simulation1.Icr_D_d,label='Icr_D sim1')
plt.plot(simulation1.t,simulation1.Ise_D_d,label='Ise_D sim1')

plt.legend(loc=0)
plt.title('Deaths by cause')
plt.show()

### Hospital saturation

In [ ]:
plt.plot(simulation2.t+tsim1,simulation2.H_sat,label='HSat sim2')
plt.plot(simulation1.t,simulation1.H_sat,label='HSat sim1')
plt.plot(simulation2.t+tsim1,simulation2.V_sat,label='VSat sim2')
plt.plot(simulation1.t,simulation1.V_sat,label='Vsat sim1')
plt.legend(loc=0)
plt.title('Saturation')
plt.show()

# Error Calculation
**No está listo =(**

The following plots will show the simulation results with the real data

### Errors

In [ ]:
simulation = simulation1
t_end_idx = np.where(np.array(RM.tr)>=t_end)[0][0]
idx = np.searchsorted(simulation.t,RM.tr[:t_end_idx])
E_I = np.sum(abs(RM.Ir[:t_end_idx]-simulation.I_det[idx]))/(np.mean(RM.Ir[:t_end_idx])*t_end_idx)

t_end_idx = np.where(np.array(RM.I_d_r_tr)>=t_end)[0][0]
idx = np.searchsorted(simulation.t,RM.I_d_r_tr[:t_end_idx])
E_Id = np.sum(abs(RM.I_d_r[:t_end_idx]-simulation.I_d_det[idx]))/(np.mean(RM.I_d_r[:t_end_idx])*t_end_idx)

t_end_idx = np.where(np.array(RM.I_ac_r_tr)>=t_end)[0][0]
idx = np.searchsorted(simulation.t,RM.I_ac_r_tr[:t_end_idx])
E_Iac = np.sum(abs(RM.I_ac_r[:t_end_idx]-simulation.I_ac_det[idx]))/(np.mean(RM.I_ac_r[:t_end_idx])*t_end_idx)

t_end_idx = np.where(np.array(RM.Br_tr)>=t_end)[0][0]
idx = np.searchsorted(simulation.t,RM.Br_tr[:t_end_idx])
E_D = np.sum(abs(RM.Br[:t_end_idx]-simulation.B[idx]))/(np.mean(RM.Br[:t_end_idx])*t_end_idx)

t_end_idx = np.where(np.array(RM.sochimi_tr)>=t_end)[0][0]
idx = np.searchsorted(simulation.t,RM.sochimi_tr[:t_end_idx])
E_UCI = np.sum(abs(RM.Hr[:t_end_idx]-(simulation.Hse[idx]+simulation.Hout[idx])))/(np.mean(RM.Hr[:t_end_idx])*t_end_idx)

t_end_idx = np.where(np.array(RM.sochimi_tr)>=t_end)[0][0]
idx = np.searchsorted(simulation.t,RM.sochimi_tr[:t_end_idx])
E_VMI = np.sum(abs(RM.Vr[:t_end_idx]-simulation.V[idx]))/(np.mean(RM.Vr[:t_end_idx])*t_end_idx)

Err = {'Active Infected':E_I,'Daily Infected':E_Id,'Accumulated Infected':E_Iac,'Deaths':E_D,'UCI/UTI':E_UCI,'VMI':E_VMI}


In [ ]:
Err = {'Active Infected':E_I,'Daily Infected':E_Id,'Accumulated Infected':E_Iac,'Deaths':E_D,'UCI/UTI':E_UCI,'VMI':E_VMI}
print(Err)

# Objective Functions



In [ ]:
# Date until which we calculate the error (normally when the quarantine dynamic change)
t1_date = datetime(2020,8,15)


t1_day = (t1_date-initdate).days
t1_idx = np.where(np.array(RM.tr)>=t1_day)[0][0]
idx = np.searchsorted(simulation.t,RM.tr[:t1_idx])

## Error:

In [ ]:
Err = np.sum(abs(RM.Ir[:t1_idx]-simulation.I[idx]))/np.mean(RM.Ir[:t1_idx])
print(Err)


In [ ]:
simulation.peak_date

In [ ]:
RM.initdate